In [ ]:
]activate ../..

In [ ]:
using Gen
using GenWorldModels

In [ ]:
include("../main.jl")
AI = AudioInference
using .AI: AudioSource, gammatonegram

In [ ]:
include("../worldmodel/smart_birth_death.jl")

In [ ]:
include("plotting_utils.jl")

In [ ]:
ground_truth, _ = generate(AI.generate_scene, AI.args, choicemap((:kernel => :n_tones, 3)))
initial_tr, weight = AI.generate_initial_tr(ground_truth, num_sources=0)
gram = AI.error_gram(initial_tr)
AI.plot_gtg(gram)

In [ ]:
initial_tr, weight = AI.generate_initial_tr(ground_truth, num_sources=0)

nothing

In [ ]:
using Serialization

In [ ]:
serialize("chociemap.txt", get_choices(inferred_tr))

In [ ]:
SOBEL_FILTER = [
    -0.2 -0.5 0 0.5 0.2;
    -0.4 -1.0 0 1.0 0.4;
    -0.2 -0.5 0 0.5 0.2
]
edges = Detector.conv(error_gram(inferred_tr), SOBEL_FILTER)
AI.plot_gtg(edges/2 .+ 50)

In [ ]:
BIG_SOBEL_FILTER = [
      -0.2 -0.2 -0.2 -0.5 0 0.5 0.2 0.2 0.2;
      -0.2 -0.2 -0.2 -0.5 0 0.5 0.2 0.2 0.2;
    -0.2 -0.2 -0.2 -0.5 0 0.5 0.2 0.2 0.2;
    -0.4  -0.4 -0.4 -1.0 0 1.0 0.4 0.4 0.4;
    -0.2 -0.2 -0.2 -0.5 0 0.5 0.2 0.2 0.2;
      -0.2 -0.2 -0.2 -0.5 0 0.5 0.2 0.2 0.2;
    -0.2 -0.2 -0.2 -0.5 0 0.5 0.2 0.2 0.2;
]
edges2 = Detector.conv(error_gram(inferred_tr), BIG_SOBEL_FILTER)
AI.plot_gtg(edges2/2 .+ 50)

In [ ]:
function get_likely_start_end(tr, ch)
    eg = error_gram(tr)
    (ysize, xsize) = size(eg)

    st = Int(floor(ch[:onset] * xsize))
    nd = Int(floor((min(ch[:onset] + ch[:duration], 1)) * xsize))

    if ch[:is_noise]
        miny, maxy = 1, ysize
    else
        meany = Int(floor(pos_for_erb_val(ch[:erb])))
        miny, maxy = meany - TONESIZE/2, meany + TONESIZE/2
    end

    eg_region = eg[miny:maxy, st:nd]
    
    (startsegs, endsegs) = Detector.get_start_end_segs(eg_region)

    selected_startseg = startsegs[argmax(map(seglength, startsegs))]
  selected_endseg = endsegs[argmax(map(seglength, endsegs))]

    (st + selected_endseg.x, st + selected_startseg.x)
end

In [ ]:
(ept, spt) = get_likely_start_end(inferred_tr, get_subtree(get_choices(inferred_tr), :world => :waves => AudioSource(1)))

In [ ]:
AI.plot_gtg(add_vert_bars(AI.error_gram(inferred_tr), [ept, spt]))

In [ ]:
[s for s in ssegs if s.ymax - s.ymin > 30]

In [ ]:
[s for s in esegs if s.ymax - s.ymin > 30]

In [ ]:
?argmax

In [ ]:
get_subtree(get_choices(inferred_tr), :kernel => :n_tones)

In [ ]:
AI.plot_gtg(100 * (edges2 .< minimum(edges2) * 0.5))

In [ ]:
AI.plot_gtg(100 * (edges2 .> maximum(edges2) * 0.3))

In [ ]:
sources = Detector.get_detected_sources(gram; threshold=0.5, scenelength=2.)
AI.plot_gtg(img_with_source_rects(gram, (sources), 2))

In [ ]:
trs = []
for _=1:10
  acc = false
  while !acc
    tr, acc = mh(initial_tr, smart_birth_death_mh_kern; check=true)
      push!(trs, tr)
  end
end

In [ ]:
for tr in trs
  AI.plot_gtg(underlying_gram(tr))
  AI.PyPlot.figure()
end

In [ ]:
Detector.MIN_SOURCE_LENGTH

In [ ]:
include("../worldmodel/inference2.jl")

In [ ]:
inferred_tr = drift_smartsmbd_inference(initial_tr, 10)
nothing

In [ ]:
AI.plot_gtg(underlying_gram(inferred_tr))

In [ ]:
new_inferred_tr = drift_smartsmbd_inference(inferred_tr, 10)
isnew = new_inferred_tr != inferred_tr
inferred_tr = new_inferred_tr
isnew

In [ ]:
AI.plot_gtg(underlying_gram(inferred_tr))

In [ ]:
AI.plot_gtg(observed_gram(inferred_tr))

In [ ]:
[
  score(inferred_tr, Death(
      source_type(inferred_tr, i), i)
    )
  for i=1:num_sources(inferred_tr)
]

In [ ]:
score(inferred_tr, Death(tone, 2
    ))

In [ ]:
score(inferred_tr, Death(tone, 4))

In [ ]:
tr, acc = metropolis_hastings(inferred_tr, smart_birth_death_mh_kern; check=false, logscores=true, logfwdchoices=true)
acc

In [ ]:
get_subtree(get_choices(inferred_tr), :world)

In [ ]:
function test_drift(idx)
  total = 0
  for _=1:1
    inf_tr, acc = mh(inferred_tr, start_drift, (idx,))
    if acc
      total += 1
    end
  end
  return total/1
end

In [ ]:
test_drift(4)

In [ ]:
tr, weight = generate(onset_drift, (inferred_tr, 4))
get_choices(tr)

In [ ]:
it, uwt, _, _ = update(inferred_tr, AI.args, map(_ -> NoChange(), AI.args), get_choices(tr))
uwt

In [ ]:
uwt

In [ ]:
AI.plot_gtg(observed_gram(inferred_tr))

In [ ]:
hard_tr = inferred_tr

In [ ]:
AI.plot_gtg(error_gram(inferred_tr))

In [ ]:
sources = Detector.get_detected_sources(error_gram(inferred_tr))
[source for source in sources if score(error_gram(tr), source) > exp(-0.2)]

In [ ]:
AI.plot_gtg(img_with_source_rects(observed_gram(inferred_tr), (sources), 2))

In [ ]:
maximum(map(s -> score(error_gram(tr), s), sources))

In [ ]:
[source for source in sources if source.is_noise]

In [ ]:
[s => score(observed_gram(tr), s) for s in sources if s.is_noise]

In [ ]:
source_to_score = [s => score(observed_gram(tr), s) for s in sources]
filtered = [s => sc for (s, sc) in source_to_score if sc > exp(1.3)]

In [ ]:
AI.plot_gtg(img_with_source_rects(gram, map(x -> x[1], filtered), 2))

In [ ]:
"""
    score(gram, s::Source; threshold=0.5)

The score of a birth action gives a heuristic estimate of how "good" it is to add this object.
This score is calculated by inspecting the rough rectangle in which the tone/noise will be placed,
and counting how many pixels in the errorgram are "on", minus how many are off.  We normalize this by dividing
by the rectangle area, to get a measure of "fraction_improvement"
(what fraction of the area is an improvement, minus the fraction that makes things worse).
The score is e^(λ * improvement).
"""
function score(gram, s::Detector.Source; threshold=0.5)
    ((miny, minx), (maxy, maxx)) = rect_for_source(gram, s, 2)
    gram = @view gram[miny:maxy, minx:maxx]
    maxval = maximum(gram)
    greater_than_threshold = gram .> maxval * threshold
    less_than_threshold = gram .< maxval * threshold
    area = (maxx - minx + 1) * (maxy - miny + 1)
    improvement = (sum(greater_than_threshold) - sum(less_than_threshold))/area
    if improvement > 1
      println("greater_than_threshold = ")
      display(greater_than_threshold)
      println("sum of greater_than_threshold = ", sum(greater_than_threshold))
      println("sum of less_than_threshold = ", sum(less_than_threshold))
      println("area = ", area)
      println("rect: ", ((miny, minx), (maxy, maxx)))
      @assert false
    end
    return exp(λ * improvement)
end

In [ ]:
tr, acc = mh(initial_tr, smart_birth_death_mh_kern; check=true)
println("acc: ", acc)

In [ ]:
AI.plot_gtg(underlying_gram(tr))

In [ ]:
total_acc = 0
for _=1:20
  tr, acc = mh(tr, smart_birth_death_mh_kern; check=true)
  total_acc += acc
end
println("Total number accepted: ", total_acc)

In [ ]:
function iterated_birth_death(tr, bd_kernel, accumulated_weight=0; check=true)
  (new_tr, weight) = bd_kernel(tr; check=check)
  accumulated_weight += weight
  stop_prob = exp(accumulated_weight) > 0.5 ? 0.99 : 0.25
  if bernoulli(stop_prob)
    return (new_tr, accumulated_weight)
  else
    return iterated_birth_death(new_tr, bd_kernel, accumulated_weight; check=check)
  end
end

In [ ]:
total_acc = 0
for _=1:20
  new_tr, weight = iterated_birth_death(tr, smart_birth_death_mh_kern; check=true)
  if bernoulli(weight)
    total_acc += 1
    tr = new_tr
  end
end
println("Total number accepted: ", total_acc)

In [ ]:
(new_tr, weight) = iterated_birth_death(tr, smart_birth_death_mh_kern; check=true)

In [ ]:
AI.plot_gtg(underlying_gram(tr))

In [ ]:
AI.plot_gtg(underlying_gram(new_tr))

In [ ]:
(proptr, _) = generate(smart_birth_death_proposal, (tr,))
get_choices(proptr)

In [ ]:
assess(smart_birth_death_proposal, (tr,), get_choices(proptr))

In [ ]:
newtr, weight, _, _ = update

In [ ]:
get_subtree(get_choices(tr), :world => :waves => AudioSource(2))

In [ ]:
(sum(underlying_gram(ground_truth)[:, 5]) - 1054)/64

In [ ]:
get_subtree(get_choices(ground_truth), :world)

In [ ]:
proptr, wt = generate(smart_birth_death_proposal, (initial_tr,), choicemap((:do_smart_birth, false)))
get_choices(proptr)

In [ ]:
first_pass_results = GenWorldModels.run_first_pass(smart_birth_death_involution, tr, proptr)
first_pass_results.update_spec.subspec

In [ ]:
newtr, wt, _, _ = update(tr, get_args(initial_tr),
map(_ -> NoChange(), get_args(initial_tr)),
first_pass_results.update_spec,
invert(first_pass_results.reverse_regenerated));
wt;

In [ ]:
wt

In [ ]:
get_subtree(get_choices(newtr), :world)

In [ ]:
wt

In [ ]:
AI.plot_gtg(underlying_gram(newtr))

In [ ]:
AI.plot_gtg(observed_gram(newtr))